In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from matplotlib import pyplot as plt
import seaborn as sns

pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

In [2]:
train_df = pd.read_csv("../input/google-machine-learning-crash-course/california_housing_train (5).xls")
test_df = pd.read_csv("../input/google-machine-learning-crash-course/california_housing_test (2).csv")

# Shuffling the data
train_df = train_df.reindex(np.random.permutation(train_df.index))

In [3]:
# Convert raw value to their Z-score
train_df_mean = train_df.mean()
train_df_std = train_df.std()
train_df_norm = (train_df - train_df_mean)/train_df_std

test_df_mean = test_df.mean()
test_df_std = test_df.std()
test_df_norm = (test_df - test_df_mean)/test_df_std

In [4]:
# This cell code creates feature layers constaining three features
feature_columns = []

# resolution in Z's
resolution_in_Zs = 0.3

# Create a bucket feature column for latitude
latitude_as_a_numeric_column = tf.feature_column.numeric_column("latitude")
latitude_boundaries = list(np.arange(int(min(train_df_norm["latitude"])),
                                     int(max(train_df_norm["latitude"])),
                                     resolution_in_Zs))
latitude = tf.feature_column.bucketized_column(latitude_as_a_numeric_column, latitude_boundaries)

# Create a bucket feature column for longitude
longitude_as_a_numeric_column = tf.feature_column.numeric_column("longitude")
longitude_boundaries = list(np.arange(int(min(train_df_norm["longitude"])),
                                      int(max(train_df_norm["longitude"])),
                                      resolution_in_Zs))
longitude = tf.feature_column.bucketized_column(longitude_as_a_numeric_column, longitude_boundaries)

# Create feature cross of latitude and longitude
latitude_x_longitude = tf.feature_column.crossed_column([latitude, longitude], hash_bucket_size=100)
cross_feature = tf.feature_column.indicator_column(latitude_x_longitude)
feature_columns.append(cross_feature)

# Represent median_income and population as floting point value
median_income = tf.feature_column.numeric_column("median_column")
feature_columns.append(median_income)

population = tf.feature_column.numeric_column("population")
feature_columns.append(population)

# Convert these feature columns into a layer
my_feature_layer = tf.keras.layers.DenseFeatures(feature_columns)